In [238]:
from postgres_connect import *
import numpy as np


##### Revenue

In [239]:
#change below for date----------------------------------------------------------------
query = '''
        select awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city from public.revenue_24_jun 
        where attributed_month = '2024-05-01'       
'''
df_revenue = get_df_from_sql('Yes',query,'tunnel-ssh.cer')
df_revenue.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
0,GS1472650591,Man matters,Delhi,110066,Delivered,Prepaid,260.0,18.0,301,Delhi
1,GS2064847292,Man matters,Delhi,110049,Delivered,Prepaid,153.0,18.0,301,Delhi
2,GS1007447048,Man matters,Delhi,110075,Delivered,Prepaid,120.0,18.0,301,Delhi
3,GS1898033811,Man matters,Delhi,110085,Delivered,Prepaid,110.0,18.0,301,Delhi
4,GS1655676780,Man matters,Delhi,110088,Delivered,Prepaid,280.0,18.0,301,Delhi


In [240]:
df_revenue_filtered = df_revenue[(df_revenue['shipper_id'] == 301) & (df_revenue['shipping_city'] != 'Pune')]
df_revenue_filtered['warehouse_city'] = df_revenue_filtered['warehouse_city'].replace('NCR', 'Delhi')
df_revenue_filtered['shipping_city'] = df_revenue_filtered['warehouse_city'].replace('NCR', 'Delhi')
df_revenue_filtered.groupby('warehouse_city').aggregate('count')

,awb,user_name,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
warehouse_city,,,,,,,,,
Bangalore,176670,176670,176670,176670,176670,176670,176670,176670,176670
Delhi,244046,244046,244046,244046,244046,244046,244046,244046,244046
Hyderabad,52180,52180,52180,52180,52180,52180,52180,52180,52180
Jaipur,344,344,344,344,344,344,344,344,344
Mumbai,84359,84359,84359,84359,84359,84359,84359,84359,84359


In [241]:
df_revenue_filtered.groupby('shipping_city').aggregate('count')

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id
shipping_city,,,,,,,,,
Bangalore,176670,176670,176670,176670,176670,176670,176670,176670,176670
Delhi,244046,244046,244046,244046,244046,244046,244046,244046,244046
Hyderabad,52180,52180,52180,52180,52180,52180,52180,52180,52180
Jaipur,344,344,344,344,344,344,344,344,344
Mumbai,84359,84359,84359,84359,84359,84359,84359,84359,84359


In [242]:
print('Delivered shape',df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered'].shape)
print('RTO Delivered shape',df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered'].shape)
print('Delivered Revenue',df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered']['shipment_charges'].sum())
print('RTO Delivered Revenue',df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered']['shipment_charges'].sum())

Delivered shape (516005, 10)
RTO Delivered shape (41598, 10)
Delivered Revenue 31775425.8005
RTO Delivered Revenue 3647106.3813


In [243]:
df_revenue_delivered = df_revenue_filtered[df_revenue_filtered['shipment_status']=='Delivered']

In [244]:
df_revenue_rto = df_revenue_filtered[df_revenue_filtered['shipment_status']=='RTO Delivered']

In [245]:
df_del_grouped = df_revenue_delivered.groupby(['shipping_city','shipment_status']).aggregate({
    'awb':'count'
}).reset_index().rename(columns={'awb':'orders'})
df_del_grouped.head()

,shipping_city,shipment_status,orders
0,Bangalore,Delivered,166606
1,Delhi,Delivered,220334
2,Hyderabad,Delivered,48581
3,Jaipur,Delivered,332
4,Mumbai,Delivered,80151


In [246]:
df_rto_grouped = df_revenue_rto.groupby(['shipping_city','shipment_status']).aggregate({
    'awb':'count'
}).reset_index().rename(columns={'awb':'orders'})
df_rto_grouped.head()

,shipping_city,shipment_status,orders
0,Bangalore,RTO Delivered,10064
1,Delhi,RTO Delivered,23712
2,Hyderabad,RTO Delivered,3599
3,Jaipur,RTO Delivered,12
4,Mumbai,RTO Delivered,4208


In [247]:
df_rto_per = df_del_grouped.merge(df_rto_grouped,on='shipping_city',how='left')
df_rto_per.head()

,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y
0,Bangalore,Delivered,166606,RTO Delivered,10064
1,Delhi,Delivered,220334,RTO Delivered,23712
2,Hyderabad,Delivered,48581,RTO Delivered,3599
3,Jaipur,Delivered,332,RTO Delivered,12
4,Mumbai,Delivered,80151,RTO Delivered,4208


In [248]:
df_rto_per['rto_per']=df_rto_per['orders_y']/(df_rto_per['orders_x']+df_rto_per['orders_y'])
df_rto_per.head()

,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per
0,Bangalore,Delivered,166606,RTO Delivered,10064,0.056965
1,Delhi,Delivered,220334,RTO Delivered,23712,0.097162
2,Hyderabad,Delivered,48581,RTO Delivered,3599,0.068973
3,Jaipur,Delivered,332,RTO Delivered,12,0.034884
4,Mumbai,Delivered,80151,RTO Delivered,4208,0.049882


##### Payout

In [249]:
df_rider_cost = pd.read_csv('Rider Payout Sheet All Months - 6_month.csv')
df_rider_cost = df_rider_cost.drop(columns=['PFESIC','Finder Fees','Commission','Vendor Pay','MIS PAY'])
df_rider_cost.head()    

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key
0,Bangalore,3637,chandu M,DS BLR HBBL,3WH,4 Wheeler,11080.0,0.0,0.0,0,0.0,11080.0,2024-01-01,Bangalore-45292-3WH
1,Bangalore,2053,DILIP KUMAR TATA ACE,DS BLR MTH,3WH,FIXED,14680.0,1295.0,0.0,0,1295.0,13385.0,2024-01-01,Bangalore-45292-3WH
2,Bangalore,2733,Manjunath R,DS BLR BOMM,3WH,FIXED,16775.0,0.0,0.0,0,0.0,16775.0,2024-01-01,Bangalore-45292-3WH
3,Bangalore,5284,Chethu,DS BLR BOMM,3WH,FIXED,4265.0,0.0,0.0,0,0.0,4265.0,2024-01-01,Bangalore-45292-3WH
4,Bangalore,2450,M Chandu,DS BLR HBBL,3WH,ORDER,5264.0,0.0,1514.0,0,1514.0,3750.0,2024-01-01,Bangalore-45292-3WH


In [250]:
#change below for date----------------------------------------------------------------
df_rider_cost_filtered = df_rider_cost[df_rider_cost['date']=='2024-05-01']


##### Vendor

In [251]:
df_vendor_cost = pd.read_csv('Vendor Pay Master - May-Rider-Vendor (2).csv')
df_vendor_cost.head()

,Key,City,Month,Vendor,Total Pay,Net Deduction,Net Pay,Negative Pay,PFESIC,Sourcing Fees,Commission,Comm. Amount,Vendor Pay,MIS PAY,MIS Amount,Dif In MIS,Diff in Invoice,Extra per rs
0,Bangalore-45413-3WH,Bangalore,01/05/2024,3WH,"91,111","22,990","68,121",0,0,0,NaN,0,68121,91111,0,NaN,NaN,0.000
1,Bangalore-45413-Blitz_Self,Bangalore,01/05/2024,Blitz_Self,"2,928","2,312",616,0,0,0,NaN,0,0,2312,0,NaN,NaN,0.000
2,Bangalore-45413-Hindusree,Bangalore,01/05/2024,Hindusree,"1,873,425","298,312","1,575,113",0,0,0,0,"78,756",1653869,1952180,1952180,₹0,NaN,0.042
3,Bangalore-45413-Peel works,Bangalore,01/05/2024,Peel works,"613,449","124,708","488,741","78,237","74,156","7,000",0,"38,468",686602,811310,694582,"-₹116,728",NaN,0.323
4,Bangalore-45413-Quess Corp,Bangalore,01/05/2024,Quess Corp,"74,143",100,"74,043",0,"10,423",0,0,"5,913",90379,90479,90379,-₹100,NaN,0.220


##### FM-MM

In [252]:
df_mm_cost = pd.read_csv('Vendor Pay Master - FM-MM.csv')
df_mm_cost.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
0,01/01/2024,Bangalore,Bangalore,NaN,"₹11,18,555",NaN,Hyperlocal
1,01/01/2024,Delhi,Bangalore,NaN,"₹2,05,699",NaN,Intercity
2,01/01/2024,Delhi,Delhi,NaN,"₹12,91,611",NaN,Hyperlocal
3,01/01/2024,Bangalore,Hyderabad,Rippler,"₹2,94,664",NaN,Intercity
4,01/01/2024,Bangalore,Hyderabad,General Travels,"₹1,03,040",NaN,Intercity


In [253]:
df_mm_cost_filtered = df_mm_cost[df_mm_cost['Month']=='01/05/2024']
def convert_currency_to_number(currency_str):
    if pd.isna(currency_str):
        return None
    return int(currency_str.replace('₹', '').replace(',', ''))

# Apply the function to the Cost column
df_mm_cost_filtered['Cost'] = df_mm_cost_filtered['Cost'].apply(convert_currency_to_number).astype(float)
df_mm_cost_filtered.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
36,01/05/2024,Delhi,Bangalore,Rippler,542837.0,NaN,Intercity
37,01/05/2024,Bangalore,Bangalore,NaN,1869930.0,NaN,Hyperlocal
38,01/05/2024,Delhi,Delhi,NaN,1868216.0,NaN,Hyperlocal
39,01/05/2024,Delhi,Hyderabad,Rippler,296703.0,NaN,Intercity
40,01/05/2024,Bangalore,Hyderabad,NaN,712478.0,NaN,Intercity


##### Fuel

In [254]:
df_fuel_cost = pd.read_csv('Vendor Pay Master - Fuel.csv')
df_fuel_cost.head()

,city,amount,Month
0,Delhi,565626,01/01/2024
1,Bangalore,599032,01/01/2024
2,Hyderabad,352854,01/01/2024
3,Mumbai,379773,01/01/2024
4,Delhi,347268,01/02/2024


In [255]:
df_fuel_cost_filtered = df_fuel_cost[df_fuel_cost['Month']=='01/05/2024']
df_fuel_cost_filtered.head()

,city,amount,Month
44,Delhi,674435,01/05/2024
45,Delhi,35412,01/05/2024
46,Jaipur,121153,01/05/2024
47,Bangalore,99855,01/05/2024
48,Bangalore,103134,01/05/2024


In [256]:
df_fuel_cost_grouped=df_fuel_cost_filtered.groupby('city').aggregate({'amount':'sum'}).reset_index()
df_fuel_cost_grouped.head()

,city,amount
0,Bangalore,331803
1,Delhi,709847
2,Hyderabad,423870
3,Jaipur,121153
4,Mumbai,367794


##### Lost on Hub

In [258]:
df_hubloss_cost = pd.read_csv('Vendor Pay Master - lostonhub (1).csv')
df_hubloss_cost.head()

,Head,Team,wh_code,Amount,Month
0,Loss on Hub,Bangalore,DS_BLR_MTH,"64,669",01/01/2024
1,Loss on Hub,Mumbai,DS_BOM_KRL,"35,581",01/01/2024
2,Loss on Hub,Hyderabad,DS_HYD_KMP,"37,712",01/01/2024
3,Loss on Hub,Delhi,DS_NCR_DWK,"76,844",01/01/2024
4,Loss on Hub,Bangalore,DS_BLR_MTH,"69,881",01/02/2024


##### Franchise

In [259]:
df_franchise_cost = pd.read_csv('Rider Payout Sheet All Months - Fran_6_month.csv')
df_franchise_cost.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
0,2024-01-01,Bangalore,BLR FRH ECTY,"556,840",0,"18,947",0,"18,947","537,893"
1,2024-01-01,Bangalore,CMRJ-Franchise,"63,640","8,974","14,798",0,"23,771","39,869"
2,2024-01-01,Bangalore,STNG-Franchise,"11,600",0,"1,910",0,"1,910","9,690"
3,2024-01-01,Bangalore,UUTR-Franchise,"325,120",198,"23,478",0,"23,676","301,444"
4,2024-01-01,Delhi,DEL FRH GKP,"667,102","13,645","64,470",0,"78,115","588,987"


In [260]:
df_franchise_cost_filtered = df_franchise_cost[df_franchise_cost['date']=='2024-05-01']
df_franchise_cost_filtered.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
55,2024-05-01,Bangalore,BLDR-Franchise,"653,215",0,"11,375","3,550","14,925","638,290"
56,2024-05-01,Bangalore,BLR FRH ECTY,"712,455","12,940","16,740","7,050","36,730","675,725"
57,2024-05-01,Bangalore,CMRJ-Franchise,"273,152",0,"1,084","3,900","4,984","268,168"
58,2024-05-01,Bangalore,STNG-Franchise,"175,650",0,0,"1,850","1,850","173,800"
59,2024-05-01,Bangalore,UUTR-Franchise,"375,946",0,"2,862","3,900","6,762","369,184"


##### Delivered

In [261]:
query_kaptaan = ''' 
    with locus as (
    select * from locus_task_brief where date_trunc('month',dispatch_time)::date = '2024-05-01'
    )
    
    select 
        COALESCE(tour.rider_id, r.rider_id) AS num_rider_id
    ,    COALESCE(t.trip_name, l.awb) as awb 
    ,   t.trip_id
    ,   coalesce(t.start_at,l.dispatch_time) as dispatch_time
    ,   coalesce(n.node_name,nl.node_name) as node_name
    ,   coalesce(n.city_name ,nl.city_name) as shipping_city
    from application_db.trip t
    left join locus as l on l.task_id = t.locus_trip_id
    left join application_db.tour on t.tour_id = tour.tour_id
    left join application_db.node n on tour.node_id = n.node_id
    left join application_db.rider as r on l.rider_id = r.locus_rider_id
    left join application_db.node nl on l.location_id = nl.locus_home_base_id

    where (t.status = 'COMPLETED' or l.status = 'COMPLETED')
    and date_trunc('month',t.start_at)::date = '2024-05-01'
'''

df_delivered_awb = get_df_from_sql('Yes',query_kaptaan,'tunnel-ssh.cer')

df_delivered_awb.head()

,num_rider_id,awb,trip_id,dispatch_time,node_name,shipping_city
0,1016.0,BZPPL00401769,4811572,2024-05-31 03:53:11.582,DS BLR MTH,Bangalore
1,1016.0,BZPPL00404111,4811626,2024-05-31 03:53:25.157,DS BLR MTH,Bangalore
2,1016.0,GS1823484502,4811542,2024-05-31 03:56:21.600,DS BLR MTH,Bangalore
3,1016.0,GS1897399591,4811568,2024-05-31 03:53:49.129,DS BLR MTH,Bangalore
4,1016.0,GS1427142835,4811546,2024-05-31 03:54:59.238,DS BLR MTH,Bangalore


In [262]:
df_delivered_awb.isna().sum()

num_rider_id     43
awb               0
trip_id           0
dispatch_time     0
node_name         0
shipping_city     0
dtype: int64

In [263]:
df_delivered_awb = df_delivered_awb.dropna()

In [264]:
df_delivered_awb['num_rider_id']=df_delivered_awb['num_rider_id'].astype(int)

In [265]:
df_delivered_city = df_delivered_awb.groupby('shipping_city').aggregate(
    {'awb':'count'
     }).reset_index()
df_delivered_city.head()

,shipping_city,awb
0,Bangalore,147549
1,Delhi,184607
2,Hyderabad,80678
3,Jaipur,18348
4,Mumbai,85464


In [266]:
df_delivered_rider = df_delivered_awb.groupby('num_rider_id').aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
df_delivered_rider.head()

,num_rider_id,orders
0,840,1446
1,845,1629
2,850,1424
3,1000,1682
4,1014,1270


In [267]:
df_delivered_franchise = df_delivered_awb.groupby(['num_rider_id','node_name']).aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
# df_delivered_node_filtered = df_delivered_franchise[df_delivered_franchise['node_name']!='BLR_OFSC']
# df_delivered_node_filtered.head()
df_delivered_franchise.head()

,num_rider_id,node_name,orders
0,840,DS BLR MTH,1446
1,845,DS BLR BOMM,1629
2,850,DS BLR MTH,1424
3,1000,BLDR-Franchise,1682
4,1014,DS BLR BOMM,1270


In [268]:
df_delivered_node = df_delivered_awb.groupby(['node_name']).aggregate(
    {'awb':'count'
     }).rename(columns={'awb': 'orders'}).reset_index()
df_delivered_node_filtered = df_delivered_node[df_delivered_node['node_name']!='BLR_OFSC']
df_delivered_node_filtered.head()
df_delivered_node_filtered.head()

,node_name,orders
0,APAN-Franchise,10922
1,BACP-Franchise,1168
2,BLDR-Franchise,17366
3,BLR FRH ECTY,19184
5,BRBN-Franchise,4272


##### Rider Pay CPO

In [269]:
df_payout = df_rider_cost_filtered.merge(df_delivered_rider, left_on='rider_id',right_on='num_rider_id', how='left').drop(columns=['num_rider_id'])
df_payout.head()

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,orders
0,Bangalore,5520,chandu raja,DS BLR HBBL,3WH,FIXED,7940.0,572.0,0.0,300,872.0,7068.0,2024-05-01,Bangalore-45413-3WH,141.0
1,Bangalore,5718,sandeep k,DS BLR HBBL,3WH,ORDER,5500.0,7658.0,0.0,500,8158.0,-2658.0,2024-05-01,Bangalore-45413-3WH,220.0
2,Bangalore,5997,murugan,DS BLR BOMM,3WH,ORDER,7228.0,0.0,0.0,100,100.0,7128.0,2024-05-01,Bangalore-45413-3WH,278.0
3,Bangalore,4673,SADIQ,DS BLR MTH,3WH,ORDER,13799.0,0.0,0.0,0,0.0,13799.0,2024-05-01,Bangalore-45413-3WH,456.0
4,Bangalore,6181,sandeep kumar,DS BLR HBBL,3WH,FIXED,2900.0,0.0,0.0,200,200.0,2700.0,2024-05-01,Bangalore-45413-3WH,42.0


In [270]:
df_payout['rider_pay_cpo'] = df_payout['total_pay']/df_payout['orders']
df_payout.head()

,city,rider_id,rider_name,hub,vendor_name,payout_structure,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay,date,Key,orders,rider_pay_cpo
0,Bangalore,5520,chandu raja,DS BLR HBBL,3WH,FIXED,7940.0,572.0,0.0,300,872.0,7068.0,2024-05-01,Bangalore-45413-3WH,141.0,56.312057
1,Bangalore,5718,sandeep k,DS BLR HBBL,3WH,ORDER,5500.0,7658.0,0.0,500,8158.0,-2658.0,2024-05-01,Bangalore-45413-3WH,220.0,25.000000
2,Bangalore,5997,murugan,DS BLR BOMM,3WH,ORDER,7228.0,0.0,0.0,100,100.0,7128.0,2024-05-01,Bangalore-45413-3WH,278.0,26.000000
3,Bangalore,4673,SADIQ,DS BLR MTH,3WH,ORDER,13799.0,0.0,0.0,0,0.0,13799.0,2024-05-01,Bangalore-45413-3WH,456.0,30.260965
4,Bangalore,6181,sandeep kumar,DS BLR HBBL,3WH,FIXED,2900.0,0.0,0.0,200,200.0,2700.0,2024-05-01,Bangalore-45413-3WH,42.0,69.047619


##### Franchise Pay CPO

In [271]:
df_franchise_cost_filtered.head()

,date,city,hub,total_pay,cod_deduction,lost_deduction,fake_deduction,total_deduction,net_pay
55,2024-05-01,Bangalore,BLDR-Franchise,"653,215",0,"11,375","3,550","14,925","638,290"
56,2024-05-01,Bangalore,BLR FRH ECTY,"712,455","12,940","16,740","7,050","36,730","675,725"
57,2024-05-01,Bangalore,CMRJ-Franchise,"273,152",0,"1,084","3,900","4,984","268,168"
58,2024-05-01,Bangalore,STNG-Franchise,"175,650",0,0,"1,850","1,850","173,800"
59,2024-05-01,Bangalore,UUTR-Franchise,"375,946",0,"2,862","3,900","6,762","369,184"


In [272]:
df_delivered_node_filtered.head()

,node_name,orders
0,APAN-Franchise,10922
1,BACP-Franchise,1168
2,BLDR-Franchise,17366
3,BLR FRH ECTY,19184
5,BRBN-Franchise,4272


In [273]:
df_franchise_payout=df_franchise_cost_filtered.merge(df_delivered_node_filtered,right_on='node_name',left_on='hub',how='left')[['node_name','orders','total_pay']]
df_franchise_payout['vendor_name'] = df_franchise_payout['node_name']
df_franchise_payout['payout_structure'] = 'ORDER'
df_franchise_payout['rider_pay_cpo'] = df_franchise_payout['total_pay'].str.replace(',', '').astype(float) / df_franchise_payout['orders']
df_franchise_payout['Key'] = df_franchise_payout['node_name']

df_franchise_payout.head()

,node_name,orders,total_pay,vendor_name,payout_structure,rider_pay_cpo,Key
0,BLDR-Franchise,17366,"653,215",BLDR-Franchise,ORDER,37.614592,BLDR-Franchise
1,BLR FRH ECTY,19184,"712,455",BLR FRH ECTY,ORDER,37.137980,BLR FRH ECTY
2,CMRJ-Franchise,6864,"273,152",CMRJ-Franchise,ORDER,39.794872,CMRJ-Franchise
3,STNG-Franchise,4455,"175,650",STNG-Franchise,ORDER,39.427609,STNG-Franchise
4,UUTR-Franchise,9447,"375,946",UUTR-Franchise,ORDER,39.795279,UUTR-Franchise


In [274]:
df_franchise_payout2 = df_franchise_payout.merge(df_delivered_franchise,on='node_name',how='left').drop(columns=['orders_x','total_pay'])
df_franchise_payout2.head()

,node_name,vendor_name,payout_structure,rider_pay_cpo,Key,num_rider_id,orders_y
0,BLDR-Franchise,BLDR-Franchise,ORDER,37.614592,BLDR-Franchise,1000,1682
1,BLDR-Franchise,BLDR-Franchise,ORDER,37.614592,BLDR-Franchise,1972,1127
2,BLDR-Franchise,BLDR-Franchise,ORDER,37.614592,BLDR-Franchise,2192,1065
3,BLDR-Franchise,BLDR-Franchise,ORDER,37.614592,BLDR-Franchise,2251,1285
4,BLDR-Franchise,BLDR-Franchise,ORDER,37.614592,BLDR-Franchise,3376,1049


##### combining both and making rider pay

In [275]:
df_cost_master = df_delivered_awb.merge(df_payout,left_on = 'num_rider_id',right_on = 'rider_id',how='left')[['num_rider_id','rider_id','awb','trip_id','node_name','shipping_city','vendor_name','payout_structure','rider_pay_cpo','Key','dispatch_time']]
df_cost_master.head()

,num_rider_id,rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time
0,1016,1016.0,BZPPL00401769,4811572,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:53:11.582
1,1016,1016.0,BZPPL00404111,4811626,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:53:25.157
2,1016,1016.0,GS1823484502,4811542,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:56:21.600
3,1016,1016.0,GS1897399591,4811568,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:53:49.129
4,1016,1016.0,GS1427142835,4811546,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:54:59.238


In [276]:
df_cost_master[df_cost_master['rider_id'].isna()]

,num_rider_id,rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time
34,6245,NaN,GS1103345394,4801884,UUTR-Franchise,Bangalore,NaN,NaN,NaN,NaN,2024-05-30 07:00:20.863810
35,6245,NaN,GS1709573941,4801905,UUTR-Franchise,Bangalore,NaN,NaN,NaN,NaN,2024-05-30 06:58:44.114487
36,6245,NaN,GS1896076871,4801895,UUTR-Franchise,Bangalore,NaN,NaN,NaN,NaN,2024-05-30 06:59:31.657798
37,6245,NaN,GS2058079600,4801888,UUTR-Franchise,Bangalore,NaN,NaN,NaN,NaN,2024-05-30 06:59:59.110061
38,6245,NaN,GS2004460098,4801885,UUTR-Franchise,Bangalore,NaN,NaN,NaN,NaN,2024-05-30 07:00:09.702990
...,...,...,...,...,...,...,...,...,...,...,...
516416,5148,NaN,GS1792572657,4826141,KPHB-Franchise,Hyderabad,NaN,NaN,NaN,NaN,2024-05-31 11:06:11.014000
516417,5148,NaN,GS1631921281,4826149,KPHB-Franchise,Hyderabad,NaN,NaN,NaN,NaN,2024-05-31 11:05:06.240000
516418,5148,NaN,GS2096343760,4826145,KPHB-Franchise,Hyderabad,NaN,NaN,NaN,NaN,2024-05-31 11:05:17.509000
516419,5148,NaN,GS1590153166,4826143,KPHB-Franchise,Hyderabad,NaN,NaN,NaN,NaN,2024-05-31 11:06:00.394000


In [277]:
df_cost_master[df_cost_master['awb']=='BZPPL00247709']

,num_rider_id,rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time
10555,1845,NaN,BZPPL00247709,4792827,BLR FRH ECTY,Bangalore,NaN,NaN,NaN,NaN,2024-05-30 04:14:41.424457


In [278]:
df_cost_master_final = df_cost_master.merge(df_franchise_payout2, on='num_rider_id', how='left', suffixes=('', '_fill')).drop(columns=['orders_y','rider_id'])

# Fill the NaN values in df_cost_master with the corresponding values from df_fill
columns_to_fill = ['vendor_name', 'payout_structure', 'rider_pay_cpo', 'Key','node_name']
for column in columns_to_fill:
    df_cost_master_final[column] = df_cost_master_final[column].fillna(df_cost_master_final[f"{column}_fill"])
df_cost_master_final = df_cost_master_final.drop(columns=[f"{column}_fill" for column in columns_to_fill])

print(df_cost_master_final.head())

   num_rider_id            awb  trip_id   node_name shipping_city vendor_name  \
0          1016  BZPPL00401769  4811572  DS BLR MTH     Bangalore  Peel works   
1          1016  BZPPL00404111  4811626  DS BLR MTH     Bangalore  Peel works   
2          1016   GS1823484502  4811542  DS BLR MTH     Bangalore  Peel works   
3          1016   GS1897399591  4811568  DS BLR MTH     Bangalore  Peel works   
4          1016   GS1427142835  4811546  DS BLR MTH     Bangalore  Peel works   

  payout_structure  rider_pay_cpo                         Key  \
0            ORDER      25.986456  Bangalore-45413-Peel works   
1            ORDER      25.986456  Bangalore-45413-Peel works   
2            ORDER      25.986456  Bangalore-45413-Peel works   
3            ORDER      25.986456  Bangalore-45413-Peel works   
4            ORDER      25.986456  Bangalore-45413-Peel works   

            dispatch_time  
0 2024-05-31 03:53:11.582  
1 2024-05-31 03:53:25.157  
2 2024-05-31 03:56:21.600  
3 2024-05-

In [279]:
df_cost_master_final.isna().sum()

num_rider_id           0
awb                    0
trip_id                0
node_name              0
shipping_city          0
vendor_name         2253
payout_structure    2253
rider_pay_cpo       2253
Key                 2253
dispatch_time          0
dtype: int64

In [280]:
df_cost_master_final.dropna(inplace=True)

##### Vendor Pay CPO

In [281]:
df_vendor_cost_filtered = df_vendor_cost[['Key','Extra per rs']]
df_cost_master2 = df_cost_master_final.merge(df_vendor_cost_filtered,on='Key',how='left')
df_cost_master2.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,Key,dispatch_time,Extra per rs
0,1016,BZPPL00401769,4811572,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:53:11.582,0.323
1,1016,BZPPL00404111,4811626,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:53:25.157,0.323
2,1016,GS1823484502,4811542,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:56:21.600,0.323
3,1016,GS1897399591,4811568,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:53:49.129,0.323
4,1016,GS1427142835,4811546,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,Bangalore-45413-Peel works,2024-05-31 03:54:59.238,0.323


In [282]:
df_cost_master2['vendor_cpo']=df_cost_master2['rider_pay_cpo']*df_cost_master2['Extra per rs']
df_cost_master2.drop(columns=['Extra per rs','Key'],inplace=True)
df_cost_master2.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo
0,1016,BZPPL00401769,4811572,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:11.582,8.393625
1,1016,BZPPL00404111,4811626,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:25.157,8.393625
2,1016,GS1823484502,4811542,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:56:21.600,8.393625
3,1016,GS1897399591,4811568,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:49.129,8.393625
4,1016,GS1427142835,4811546,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:54:59.238,8.393625


In [283]:
df_cost_master2.isna().sum()

num_rider_id             0
awb                      0
trip_id                  0
node_name                0
shipping_city            0
vendor_name              0
payout_structure         0
rider_pay_cpo            0
dispatch_time            0
vendor_cpo          132742
dtype: int64

In [284]:
df_cost_master2=df_cost_master2.fillna('0')

In [285]:
df_cost_master2.isna().sum()

num_rider_id        0
awb                 0
trip_id             0
node_name           0
shipping_city       0
vendor_name         0
payout_structure    0
rider_pay_cpo       0
dispatch_time       0
vendor_cpo          0
dtype: int64

##### Fuel CPO

In [286]:

df_fuel_pay = df_delivered_city.merge(df_fuel_cost_grouped,left_on='shipping_city',right_on='city',how='left')
df_fuel_pay.head()

,shipping_city,awb,city,amount
0,Bangalore,147549,Bangalore,331803.0
1,Delhi,184607,Delhi,709847.0
2,Hyderabad,80678,Hyderabad,423870.0
3,Jaipur,18348,Jaipur,121153.0
4,Mumbai,85464,Mumbai,367794.0


In [287]:
df_fuel_pay['fuel_cpo']=df_fuel_pay['amount']/df_fuel_pay['awb']
df_fuel_pay.drop(columns=['amount','awb','city'],inplace=True)

In [288]:
df_cost_master3 = df_cost_master2.merge(df_fuel_pay,on='shipping_city',how='left')
df_cost_master3.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,1016,BZPPL00401769,4811572,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:11.582,8.393625,2.248765
1,1016,BZPPL00404111,4811626,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:25.157,8.393625,2.248765
2,1016,GS1823484502,4811542,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:56:21.600,8.393625,2.248765
3,1016,GS1897399591,4811568,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:49.129,8.393625,2.248765
4,1016,GS1427142835,4811546,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:54:59.238,8.393625,2.248765


In [289]:
df_cost_master3.isna().sum()

num_rider_id         0
awb                  0
trip_id              0
node_name            0
shipping_city        0
vendor_name          0
payout_structure     0
rider_pay_cpo        0
dispatch_time        0
vendor_cpo           0
fuel_cpo            15
dtype: int64

In [290]:
df_cost_master3=df_cost_master3.fillna(0)

In [291]:
df_cost_master3.shape

(515852, 11)

##### Merging with revenue

Note we need to attribute fm-mm cost to RTO Delivered also, so we need to get RTO orders from revenue

In [292]:
df_revenue_filtered.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city
0,GS1472650591,Man matters,Delhi,110066,Delivered,Prepaid,260.0,18.0,301,Delhi
1,GS2064847292,Man matters,Delhi,110049,Delivered,Prepaid,153.0,18.0,301,Delhi
2,GS1007447048,Man matters,Delhi,110075,Delivered,Prepaid,120.0,18.0,301,Delhi
3,GS1898033811,Man matters,Delhi,110085,Delivered,Prepaid,110.0,18.0,301,Delhi
4,GS1655676780,Man matters,Delhi,110088,Delivered,Prepaid,280.0,18.0,301,Delhi


In [293]:
df_cost_master3.head()

,num_rider_id,awb,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,1016,BZPPL00401769,4811572,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:11.582,8.393625,2.248765
1,1016,BZPPL00404111,4811626,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:25.157,8.393625,2.248765
2,1016,GS1823484502,4811542,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:56:21.600,8.393625,2.248765
3,1016,GS1897399591,4811568,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:53:49.129,8.393625,2.248765
4,1016,GS1427142835,4811546,DS BLR MTH,Bangalore,Peel works,ORDER,25.986456,2024-05-31 03:54:59.238,8.393625,2.248765


In [294]:
# df_margin = df_revenue_filtered.merge(df_cost_master3,on='awb',how='outer')
df_margin = df_revenue_filtered.merge(df_cost_master3,on='awb',how='outer').drop(columns=['shipping_city_y']).rename(columns={'shipping_city_x':'shipping_city'})
df_margin.head()

,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,shipping_city,num_rider_id,trip_id,node_name,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo
0,BZPPL00151722,Purplle,Bangalore,560099,Delivered,COD,189.0,35.0,301.0,Bangalore,5510.0,4211119.0,BLR FRH ECTY,BLR FRH ECTY,ORDER,37.137980,2024-05-03 04:29:59.330,0,2.248765
1,BZPPL00152109,Purplle,Delhi,110008,Delivered,COD,670.0,35.0,301.0,Delhi,3302.0,4355617.0,DS DEL RNGR,PS Manpower solutions,ORDER,28.660022,2024-05-09 06:36:52.262,1.146401,3.845179
2,BZPPL00189277,Purplle,Hyderabad,500018,Delivered,Prepaid,924.0,40.0,301.0,Hyderabad,5842.0,4273766.0,BRBN-Franchise,BRBN-Franchise,ORDER,39.076779,2024-05-06 03:39:13.513,0,5.253849
3,BZPPL00189434,Purplle,Hyderabad,500050,Delivered,Prepaid,189.0,40.0,301.0,Hyderabad,5631.0,4221401.0,DS HYD SRNG,Shiva Shakti,FIXED,58.867925,2024-05-03 07:20:01.047,2.590189,5.253849
4,BZPPL00192721,Purplle,Delhi,121003,Delivered,COD,1487.2,40.0,301.0,Delhi,3940.0,4311563.0,FABD-Franchise,FABD-Franchise,ORDER,36.052451,2024-05-07 07:46:18.805,0,3.845179


In [295]:
columns_to_update = [
    'num_rider_id', 'trip_id', 'node_name', 'vendor_name', 
    'payout_structure', 'rider_pay_cpo', 'dispatch_time', 
    'vendor_cpo', 'fuel_cpo'
]
df_margin.loc[df_margin['shipment_status'] == 'RTO Delivered', columns_to_update] = 0

In [296]:
df_margin.isna().sum()

awb                     0
user_name              28
warehouse_city         32
drop_pincode           28
shipment_status        28
mop                    28
chargeable_weight      28
shipment_charges       28
shipper_id             28
shipping_city          32
num_rider_id         1992
trip_id              1992
node_name            1992
vendor_name          1992
payout_structure     1992
rider_pay_cpo        1992
dispatch_time        1992
vendor_cpo           1992
fuel_cpo             1992
dtype: int64

In [297]:
df_margin = df_margin.dropna()

##### MM Costing

In [298]:
df_mm_cost_filtered.head()

,Month,Warehouse City,Shipping City,Vendor Name,Cost,Orders (computable),Lane Type
36,01/05/2024,Delhi,Bangalore,Rippler,542837.0,NaN,Intercity
37,01/05/2024,Bangalore,Bangalore,NaN,1869930.0,NaN,Hyperlocal
38,01/05/2024,Delhi,Delhi,NaN,1868216.0,NaN,Hyperlocal
39,01/05/2024,Delhi,Hyderabad,Rippler,296703.0,NaN,Intercity
40,01/05/2024,Bangalore,Hyderabad,NaN,712478.0,NaN,Intercity


In [299]:
mm_pay=df_mm_cost_filtered.groupby('Shipping City').aggregate({'Cost':'sum'}).reset_index()
mm_pay

,Shipping City,Cost
0,Bangalore,2412767.0
1,Delhi,1868216.0
2,Hyderabad,1297763.0
3,Jaipur,154807.0
4,Mumbai,953854.0


##### starting rto delivered fm

In [300]:
mm_payout_rto = mm_pay.merge(df_rto_per,right_on='shipping_city',left_on='Shipping City',how='left')
mm_payout_rto.head()

,Shipping City,Cost,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per
0,Bangalore,2412767.0,Bangalore,Delivered,166606,RTO Delivered,10064,0.056965
1,Delhi,1868216.0,Delhi,Delivered,220334,RTO Delivered,23712,0.097162
2,Hyderabad,1297763.0,Hyderabad,Delivered,48581,RTO Delivered,3599,0.068973
3,Jaipur,154807.0,Jaipur,Delivered,332,RTO Delivered,12,0.034884
4,Mumbai,953854.0,Mumbai,Delivered,80151,RTO Delivered,4208,0.049882


In [301]:
mm_payout_rto['r_factor']=mm_payout_rto['Cost']/(mm_payout_rto['orders_x']+2*mm_payout_rto['orders_y']).astype(float)
mm_payout_rto['r_factor'] = mm_payout_rto['r_factor'].round(2)
mm_payout_rto.head()

,Shipping City,Cost,shipping_city,shipment_status_x,orders_x,shipment_status_y,orders_y,rto_per,r_factor
0,Bangalore,2412767.0,Bangalore,Delivered,166606,RTO Delivered,10064,0.056965,12.92
1,Delhi,1868216.0,Delhi,Delivered,220334,RTO Delivered,23712,0.097162,6.98
2,Hyderabad,1297763.0,Hyderabad,Delivered,48581,RTO Delivered,3599,0.068973,23.27
3,Jaipur,154807.0,Jaipur,Delivered,332,RTO Delivered,12,0.034884,434.85
4,Mumbai,953854.0,Mumbai,Delivered,80151,RTO Delivered,4208,0.049882,10.77


In [302]:
margin2 = df_margin.merge(mm_payout_rto[['shipping_city','r_factor']], on='shipping_city',how='left').reset_index()

In [303]:
margin2['fm_mm_cpo'] = np.where(
    margin2['shipment_status'] == 'RTO Delivered',
    2 * margin2['r_factor'],
    np.where(
        margin2['shipment_status'] == 'Delivered',
        margin2['r_factor'],
        None  # Or any default value you prefer
    )
)
margin2.head()

,index,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,...,trip_id,node_name,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo,r_factor,fm_mm_cpo
0,0,BZPPL00151722,Purplle,Bangalore,560099,Delivered,COD,189.0,35.0,301.0,...,4211119.0,BLR FRH ECTY,BLR FRH ECTY,ORDER,37.137980,2024-05-03 04:29:59.330000,0,2.248765,12.92,12.92
1,1,BZPPL00152109,Purplle,Delhi,110008,Delivered,COD,670.0,35.0,301.0,...,4355617.0,DS DEL RNGR,PS Manpower solutions,ORDER,28.660022,2024-05-09 06:36:52.262000,1.146401,3.845179,6.98,6.98
2,2,BZPPL00189277,Purplle,Hyderabad,500018,Delivered,Prepaid,924.0,40.0,301.0,...,4273766.0,BRBN-Franchise,BRBN-Franchise,ORDER,39.076779,2024-05-06 03:39:13.513000,0,5.253849,23.27,23.27
3,3,BZPPL00189434,Purplle,Hyderabad,500050,Delivered,Prepaid,189.0,40.0,301.0,...,4221401.0,DS HYD SRNG,Shiva Shakti,FIXED,58.867925,2024-05-03 07:20:01.047000,2.590189,5.253849,23.27,23.27
4,4,BZPPL00192721,Purplle,Delhi,121003,Delivered,COD,1487.2,40.0,301.0,...,4311563.0,FABD-Franchise,FABD-Franchise,ORDER,36.052451,2024-05-07 07:46:18.805000,0,3.845179,6.98,6.98


In [304]:
margin2.isna().sum()

index                0
awb                  0
user_name            0
warehouse_city       0
drop_pincode         0
shipment_status      0
mop                  0
chargeable_weight    0
shipment_charges     0
shipper_id           0
shipping_city        0
num_rider_id         0
trip_id              0
node_name            0
vendor_name          0
payout_structure     0
rider_pay_cpo        0
dispatch_time        0
vendor_cpo           0
fuel_cpo             0
r_factor             0
fm_mm_cpo            0
dtype: int64

In [305]:
margin3 = margin2.drop(columns=['index','user_name','chargeable_weight','shipper_id','r_factor'])
margin3.rename(columns={
    'shipment_charges':'rpo','payout_structure':'rider_payout_structure','fuel_cpo':'rider_fuel_cpo'
})
margin3.head()

,awb,warehouse_city,drop_pincode,shipment_status,mop,shipment_charges,shipping_city,num_rider_id,trip_id,node_name,vendor_name,payout_structure,rider_pay_cpo,dispatch_time,vendor_cpo,fuel_cpo,fm_mm_cpo
0,BZPPL00151722,Bangalore,560099,Delivered,COD,35.0,Bangalore,5510.0,4211119.0,BLR FRH ECTY,BLR FRH ECTY,ORDER,37.137980,2024-05-03 04:29:59.330000,0,2.248765,12.92
1,BZPPL00152109,Delhi,110008,Delivered,COD,35.0,Delhi,3302.0,4355617.0,DS DEL RNGR,PS Manpower solutions,ORDER,28.660022,2024-05-09 06:36:52.262000,1.146401,3.845179,6.98
2,BZPPL00189277,Hyderabad,500018,Delivered,Prepaid,40.0,Hyderabad,5842.0,4273766.0,BRBN-Franchise,BRBN-Franchise,ORDER,39.076779,2024-05-06 03:39:13.513000,0,5.253849,23.27
3,BZPPL00189434,Hyderabad,500050,Delivered,Prepaid,40.0,Hyderabad,5631.0,4221401.0,DS HYD SRNG,Shiva Shakti,FIXED,58.867925,2024-05-03 07:20:01.047000,2.590189,5.253849,23.27
4,BZPPL00192721,Delhi,121003,Delivered,COD,40.0,Delhi,3940.0,4311563.0,FABD-Franchise,FABD-Franchise,ORDER,36.052451,2024-05-07 07:46:18.805000,0,3.845179,6.98


In [306]:
margin3.to_csv('margin_may.csv')

##### Vendor Level Margin

In [307]:
vendor_margins=margin2.groupby(['shipping_city','vendor_name','num_rider_id']).aggregate({
    'awb': 'count',
    'rider_pay_cpo': 'sum',
    'vendor_cpo': 'sum',
    'fuel_cpo': 'sum',
    'mm_cpo': 'sum',
    'shipment_charges':'sum'
}).reset_index()
vendor_margins.head()

KeyError: "Column(s) ['mm_cpo'] do not exist"

In [ ]:
vendor_margins.dtypes

shipping_city        object
vendor_name          object
num_rider_id        float64
awb                   int64
rider_pay_cpo       float64
vendor_cpo           object
fuel_cpo            float64
mm_cpo              float64
shipment_charges    float64
dtype: object

In [ ]:
vendor_margins['vendor_cpo'] = vendor_margins['vendor_cpo'].astype(float)

In [ ]:

vendor_margins['total_cost']=vendor_margins['rider_pay_cpo']+vendor_margins['vendor_cpo']+vendor_margins['fuel_cpo']+vendor_margins['mm_cpo']
vendor_margins['margin']=vendor_margins['shipment_charges']-vendor_margins['total_cost']
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay_cpo,vendor_cpo,fuel_cpo,mm_cpo,shipment_charges,total_cost,margin
0,Bangalore,3WH,4673.0,456,13799.0,0.0,1025.436757,7462.368431,65023.14205,22286.805188,42736.336862
1,Bangalore,3WH,5204.0,656,18259.0,0.0,1475.189720,10735.337041,84154.83730,30469.526761,53685.310539
2,Bangalore,3WH,5520.0,141,7940.0,0.0,317.075839,2307.442870,14507.01500,10564.518709,3942.496291
3,Bangalore,3WH,5685.0,212,5611.0,0.0,476.738141,3469.346727,24104.54350,9557.084868,14547.458632
4,Bangalore,3WH,5718.0,220,5500.0,0.0,494.728260,3600.265471,34768.63590,9594.993731,25173.642169


In [ ]:
vendor_margins.rename(columns={'fuel_cpo':'fuel_pay','mm_cpo':'mm_pay','vendor_cpo':'vendor_pay','rider_pay_cpo':'rider_pay'},inplace=True)

vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay,vendor_pay,fuel_pay,mm_pay,shipment_charges,total_cost,margin
0,Bangalore,3WH,4673.0,456,13799.0,0.0,1025.436757,7462.368431,65023.14205,22286.805188,42736.336862
1,Bangalore,3WH,5204.0,656,18259.0,0.0,1475.189720,10735.337041,84154.83730,30469.526761,53685.310539
2,Bangalore,3WH,5520.0,141,7940.0,0.0,317.075839,2307.442870,14507.01500,10564.518709,3942.496291
3,Bangalore,3WH,5685.0,212,5611.0,0.0,476.738141,3469.346727,24104.54350,9557.084868,14547.458632
4,Bangalore,3WH,5718.0,220,5500.0,0.0,494.728260,3600.265471,34768.63590,9594.993731,25173.642169


In [ ]:
vendor_margins.to_csv('vendor_margins_may.csv')

In [ ]:
vendor_margins.head()

,shipping_city,vendor_name,num_rider_id,awb,rider_pay,vendor_pay,fuel_pay,mm_pay,shipment_charges,total_cost,margin
0,Bangalore,3WH,4673.0,456,13799.0,0.0,1025.436757,7462.368431,65023.14205,22286.805188,42736.336862
1,Bangalore,3WH,5204.0,656,18259.0,0.0,1475.189720,10735.337041,84154.83730,30469.526761,53685.310539
2,Bangalore,3WH,5520.0,141,7940.0,0.0,317.075839,2307.442870,14507.01500,10564.518709,3942.496291
3,Bangalore,3WH,5685.0,212,5611.0,0.0,476.738141,3469.346727,24104.54350,9557.084868,14547.458632
4,Bangalore,3WH,5718.0,220,5500.0,0.0,494.728260,3600.265471,34768.63590,9594.993731,25173.642169


In [ ]:
margin2.head()

,index,awb,user_name,warehouse_city,drop_pincode,shipment_status,mop,chargeable_weight,shipment_charges,shipper_id,num_rider_id,trip_id,node_name,shipping_city,vendor_name,payout_structure,rider_pay_cpo,vendor_cpo,fuel_cpo,mm_cpo
0,0,BZPPL00151722,Purplle,Bangalore,560099,Delivered,COD,189.0,35.0,301.0,5510.0,4211119.0,BLR FRH ECTY,Bangalore,BLR FRH ECTY,ORDER,37.137980,0,2.248765,16.364843
1,1,BZPPL00152109,Purplle,Delhi,110008,Delivered,COD,670.0,35.0,301.0,3302.0,4355617.0,DS DEL RNGR,Delhi,PS Manpower solutions,ORDER,28.660022,1.146401,3.845179,10.132917
2,2,BZPPL00189277,Purplle,Hyderabad,500018,Delivered,Prepaid,924.0,40.0,301.0,5842.0,4273766.0,BRBN-Franchise,Hyderabad,BRBN-Franchise,ORDER,39.076779,0,5.253849,16.086309
3,3,BZPPL00189434,Purplle,Hyderabad,500050,Delivered,Prepaid,189.0,40.0,301.0,5631.0,4221401.0,DS HYD SRNG,Hyderabad,Shiva Shakti,FIXED,58.867925,2.590189,5.253849,16.086309
4,4,BZPPL00192721,Purplle,Delhi,121003,Delivered,COD,1487.2,40.0,301.0,3940.0,4311563.0,FABD-Franchise,Delhi,FABD-Franchise,ORDER,36.052451,0,3.845179,10.132917
